**Montar(Mount) Azure Data Lake mediante Service Principal**
- 1.- Obtener el client_id, tenent_id y secret_i del key vault
- 2.- Configurar Spark con APP/Client ID, Directory/Tenant Id & Secret
- 3.- Utilizar el método "mount" de "utility" para montar el almacenamiento
- 4.- Explorar otras utilidades del sistema de archivos relacionados con el montaje (list all mounts, unmounts)

In [0]:
client_id=dbutils.secrets.get(scope="movie-history-secret-scope",key="client-id")
directory_id=dbutils.secrets.get(scope="movie-history-secret-scope",key="directory-id")
client_secret=dbutils.secrets.get(scope="movie-history-secret-scope",key="client-secret")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}

In [0]:
# Unmount if already mounted  -> como lo había montado, tengo de "desmontarlo" para volver a montarlo en la ejecución
mount_point = "/mnt/mymoviehistory/demo"
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

In [0]:
# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://demo@mymoviehistory.dfs.core.windows.net/",
  mount_point = "/mnt/mymoviehistory/demo",    #mnt + nombre de cuenta de almacenamiento + nombre del contenedor
  extra_configs = configs)   # si devuelve true, es que ha creado el punto de montaje para consultarlo y poder sustituir urls largas por el punto de montaje en la celda inferior

In [0]:
#display(dbutils.fs.ls("abfss://demo@mymoviehistory.dfs.core.windows.net"))   #nombre de contenedor + nombre cuenta de almacenamiento
display(dbutils.fs.ls("/mnt/mymoviehistory/demo"))   #nombre de contenedor + nombre cuenta de almacenamiento

In [0]:
display(spark.read.csv("/mnt/mymoviehistory/demo/movie.csv"))

In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.fs.unmount("/mnt/mymoviehistory/demo")